# YOLO for Detection
For a more robust approach, we use a pre-trained YOLO model and fine-tune it for license plate detection.

YOLO (You Only Look Once) and EfficientDet are object detection algorithms that can detect multiple objects in a single pass through the network, making them much faster and more efficient than traditional approaches. This implementation uses a pre-trained EfficientDet model from TensorFlow Hub.

The advantages of this approach include:
1. **Better accuracy**: These models are designed specifically for object detection and perform well on various object sizes
2. **Speed**: They are optimized for efficient detection, making them suitable for real-time applications
3. **Robustness**: They can handle multiple license plates in a single image

To complete this implementation, you would need to:
1. Filter the detection results to keep only license plates (likely by training the model to recognize the "license plate" class)
2. Apply a confidence threshold to remove low-confidence detections
3. Post-process the bounding boxes if needed (e.g., remove overlapping detections)



In [1]:
# Import required libraries
import os
from pathlib import Path
import pandas as pd
import cv2
import matplotlib.pyplot as plt
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tqdm import tqdm
from sklearn.model_selection import train_test_split
import random
import sys

# Check if running in Colab or local
import importlib.util
IN_COLAB = importlib.util.find_spec("google.colab") is not None

# Set up paths similar to your previous notebook
current_dir = os.getcwd()
print(f"Current directory: {current_dir}")

if IN_COLAB:
    # Add project root to path to ensure imports work correctly
    project_root = os.path.join(current_dir, "Car-plate-detection")
    sys.path.insert(0, project_root)
    print(f"Project root added to path: {project_root}")
    DATA_PATH = Path(project_root+"/Data/Total")
    print(DATA_PATH)
else:
    # If not in Colab, set the project root to the current working directory's parent
    project_root = Path(os.getcwd()).parent
    print(f"Project root: {project_root}")
    DATA_PATH = project_root / "Data" / "Total"
    print(f"Data path: {DATA_PATH}")

# Install required packages for YOLO implementation
!pip install ultralytics -q  # Use ultralytics for YOLOv8

# Import after installation
from ultralytics import YOLO

Current directory: c:\ULB\MA1\Proj\PROJ-H419\Car-plate-detection\Notebooks
Project root: c:\ULB\MA1\Proj\PROJ-H419\Car-plate-detection
Data path: c:\ULB\MA1\Proj\PROJ-H419\Car-plate-detection\Data\Total



[notice] A new release of pip is available: 25.0.1 -> 25.1
[notice] To update, run: C:\ULB\MA1\Proj\PROJ-H419\proj-h419-lisence_plates_car_detection\Code\.venv\Scripts\python.exe -m pip install --upgrade pip
ERROR: Invalid requirement: '#': Expected package name at the start of dependency specifier
    #
    ^


ModuleNotFoundError: No module named 'ultralytics'

In [ ]:
# Constants
IMAGE_SIZE = (640, 640)  # Standard YOLO input size (larger than previous CNN)

# Function to load dataset similar to the previous notebook
def load_license_plate_dataset(data_path):
    # Prepare a list to collect the dataset records
    dataset = []

    # Check if data_path exists
    if not data_path.exists():
        raise FileNotFoundError(f"DATA_PATH does not exist: {data_path}\n"
                                "Please check the path or create the folder and add your data.")

    # Loop through all files in the folder
    for file in data_path.iterdir():
        if file.suffix == ".txt":
            try:
                with open(file, 'r') as f:
                    line = f.readline().strip()
                    parts = line.split('\t')

                    if len(parts) != 6:
                        print(f"Skipping malformed file: {file.name}")
                        continue

                    img_name, x, y, w, h, plate_text = parts
                    img_path = data_path / img_name

                    if not img_path.exists():
                        print(f"Image not found for annotation: {img_name}")
                        continue

                    dataset.append({
                        "image_path": str(img_path),
                        "x": int(x),
                        "y": int(y),
                        "w": int(w),
                        "h": int(h),
                        "plate_text": plate_text
                    })
            except Exception as e:
                print(f"Error processing {file}: {e}")

    # Convert to DataFrame
    df = pd.DataFrame(dataset)
    print(f"Loaded {len(df)} annotated images.")
    return df

# Load the dataset
df = load_license_plate_dataset(DATA_PATH)

# Display a sample
sample = df.iloc[random.randint(0, len(df)-1)]
img = cv2.imread(sample["image_path"])
img_rgb = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)

# Draw bounding box
x, y, w, h = sample["x"], sample["y"], sample["w"], sample["h"]
cv2.rectangle(img_rgb, (x, y), (x + w, y + h), (0, 255, 0), 2)

plt.figure(figsize=(10, 8))
plt.imshow(img_rgb)
plt.title(f"Plate: {sample['plate_text']}")
plt.axis('off')
plt.show()

In [ ]:
# Create YOLO compatible dataset structure
def create_yolo_dataset(df, output_dir, split_ratio=0.8):
    """
    Create YOLO compatible dataset structure.
    YOLO format expects:
    - images/train, images/val folders for images
    - labels/train, labels/val folders for labels
    - Each label is a .txt file with format: class_id x_center y_center width height (normalized [0-1])
    """
    # Create directory structure
    output_path = Path(output_dir)
    
    # Create directories
    (output_path / 'images' / 'train').mkdir(parents=True, exist_ok=True)
    (output_path / 'images' / 'val').mkdir(parents=True, exist_ok=True)
    (output_path / 'labels' / 'train').mkdir(parents=True, exist_ok=True)
    (output_path / 'labels' / 'val').mkdir(parents=True, exist_ok=True)
    
    # Split the dataset
    train_df, val_df = train_test_split(df, train_size=split_ratio, random_state=42)
    
    # Process training data
    for idx, row in tqdm(train_df.iterrows(), total=len(train_df), desc="Processing training data"):
        process_yolo_sample(row, output_path / 'images' / 'train', output_path / 'labels' / 'train')
    
    # Process validation data
    for idx, row in tqdm(val_df.iterrows(), total=len(val_df), desc="Processing validation data"):
        process_yolo_sample(row, output_path / 'images' / 'val', output_path / 'labels' / 'val')
    
    # Create dataset.yaml file for YOLO training
    yaml_content = f"""path: {output_path.absolute()}  # dataset root dir
train: images/train  # train images relative to path
val: images/val  # val images relative to path

nc: 1  # number of classes
names: ['license_plate']  # class names
"""
    
    with open(output_path / 'dataset.yaml', 'w') as f:
        f.write(yaml_content)
    
    print(f"YOLO dataset created at {output_path.absolute()}")
    print(f"Training samples: {len(train_df)}, Validation samples: {len(val_df)}")
    print(f"Dataset config saved to {output_path / 'dataset.yaml'}")
    
    return str(output_path / 'dataset.yaml')

def process_yolo_sample(row, images_dir, labels_dir):
    """
    Process a single sample to YOLO format.
    """
    # Get image path and read image
    img_path = Path(row["image_path"])
    img = cv2.imread(str(img_path))
    if img is None:
        print(f"Could not read image at {img_path}")
        return
    
    height, width = img.shape[:2]
    
    # Calculate normalized coordinates (center_x, center_y, width, height)
    x_center = (row["x"] + row["w"] / 2) / width
    y_center = (row["y"] + row["h"] / 2) / height
    norm_width = row["w"] / width
    norm_height = row["h"] / height
    
    # Clip values to ensure they're between 0 and 1
    x_center = max(0, min(1, x_center))
    y_center = max(0, min(1, y_center))
    norm_width = max(0, min(1, norm_width))
    norm_height = max(0, min(1, norm_height))
    
    # Create label file (class_id center_x center_y width height)
    label_content = f"0 {x_center} {y_center} {norm_width} {norm_height}\n"
    
    # Copy image to dataset folder
    new_img_path = images_dir / img_path.name
    cv2.imwrite(str(new_img_path), img)
    
    # Write label file
    label_path = labels_dir / f"{img_path.stem}.txt"
    with open(label_path, 'w') as f:
        f.write(label_content)

# Create a YOLO compatible dataset from our data
yolo_dataset_path = create_yolo_dataset(df, os.path.join(project_root, "yolo_dataset"))

# Step 2: YOLO Model Training and Fine-Tuning

Now we'll use a pre-trained YOLO model and fine-tune it specifically for license plate detection. YOLOv8 is one of the latest and most performant object detection models available, and we'll leverage its power for our task.

Key advantages of using YOLO over the custom CNN approach:

1. **Transfer Learning**: Leverages knowledge from large-scale pre-training on diverse object categories
2. **Better Feature Extraction**: Advanced backbone architecture designed specifically for object detection
3. **Multiple Scale Detection**: Better detection of license plates regardless of size in the image
4. **Robust to Occlusion**: Can detect partially visible license plates
5. **Higher Precision**: Better localization of object boundaries

In [ ]:
# Define and train the YOLO model
def train_yolo_model(dataset_yaml, epochs=50):
    """
    Train a YOLOv8 model for license plate detection.
    """
    # Load a pre-trained YOLO model (n = nano, s = small, m = medium, l = large, x = extra large)
    model = YOLO('yolov8n.pt')  # Start with smaller model for efficiency
    
    # Train the model
    results = model.train(
        data=dataset_yaml,
        epochs=epochs,
        imgsz=640,
        batch=16,
        patience=15,  # Early stopping patience
        verbose=True,
        device='0' if tf.test.is_gpu_available() else 'cpu',  # Use GPU if available
        project=os.path.join(project_root, "yolo_runs"),
        name="license_plate_detector",
        pretrained=True,
        optimizer="Adam",
        lr0=0.001,
        lrf=0.01,
        save=True,
        plots=True  # Generate training plots
    )
    
    return model

# Train the model
try:
    print("Starting YOLOv8 training...")
    yolo_model = train_yolo_model(yolo_dataset_path, epochs=30)
    print("Training complete!")
except Exception as e:
    print(f"Error during training: {e}")

In [ ]:
# Evaluate YOLO model
def evaluate_yolo_model(model, validation_folder):
    """
    Evaluate the YOLO model on the validation set.
    """
    # Run validation
    val_results = model.val(data=validation_folder)
    
    # Extract metrics
    metrics = val_results.box.metrics.dict
    print(f"mAP50: {metrics['map50']:.4f}")
    print(f"mAP50-95: {metrics['map']:.4f}")
    print(f"Precision: {metrics['precision']:.4f}")
    print(f"Recall: {metrics['recall']:.4f}")
    
    return metrics

# Run evaluation
print("Evaluating model...")
eval_metrics = evaluate_yolo_model(yolo_model, yolo_dataset_path)

# Function to calculate IoU same as in your previous notebook
def calculate_iou(box1, box2):
    """
    Calculate IoU between two boxes [x1, y1, x2, y2] format
    """
    # Extract coordinates
    x1_1, y1_1, x2_1, y2_1 = box1
    x1_2, y1_2, x2_2, y2_2 = box2
    
    # Calculate intersection area
    x1_i = max(x1_1, x1_2)
    y1_i = max(y1_1, y1_2)
    x2_i = min(x2_1, x2_2)
    y2_i = min(y2_1, y2_2)
    
    if x2_i < x1_i or y2_i < y1_i:
        return 0.0
    
    intersection_area = (x2_i - x1_i) * (y2_i - y1_i)
    
    # Calculate union area
    box1_area = (x2_1 - x1_1) * (y2_1 - y1_1)
    box2_area = (x2_2 - x1_2) * (y2_2 - y1_2)
    union_area = box1_area + box2_area - intersection_area
    
    # Calculate IoU
    iou = intersection_area / union_area if union_area > 0 else 0
    
    return iou

# Function to detect license plates in new images (similar to what you had before)
def detect_license_plate_yolo(model, image_path, conf_threshold=0.25):
    """
    Detect license plates in an image using the fine-tuned YOLO model.
    """
    # Load the image
    img = cv2.imread(image_path)
    if img is None:
        print(f"Could not read image at {image_path}")
        return None, []
    
    img_rgb = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    
    # Run inference
    results = model.predict(img_rgb, conf=conf_threshold, verbose=False)[0]
    
    # Get detection boxes
    boxes = results.boxes.xyxy.cpu().numpy()  # Get boxes in [x1, y1, x2, y2] format
    confidences = results.boxes.conf.cpu().numpy()
    
    # Draw results on image
    result_img = img_rgb.copy()
    for i, box in enumerate(boxes):
        x1, y1, x2, y2 = map(int, box)
        confidence = confidences[i]
        
        # Draw rectangle
        cv2.rectangle(result_img, (x1, y1), (x2, y2), (0, 255, 0), 2)
        
        # Add confidence text
        cv2.putText(result_img, f"{confidence:.2f}", (x1, y1-10),
                   cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 2)
    
    return result_img, boxes, confidences

# Test the detector on a few images
def test_detection_on_samples(model, df, num_samples=5):
    """
    Test the YOLO detector on random samples and show results
    """
    # Get random samples
    sample_indices = random.sample(range(len(df)), num_samples)
    
    for idx in sample_indices:
        row = df.iloc[idx]
        img_path = row["image_path"]
        
        # Ground truth box (x1, y1, x2, y2 format)
        true_box = [row["x"], row["y"], row["x"] + row["w"], row["y"] + row["h"]]
        
        # Run detection
        result_img, pred_boxes, confidences = detect_license_plate_yolo(model, img_path)
        
        # Calculate IoU if any detection is found
        iou_value = 0
        if len(pred_boxes) > 0:
            # Get the box with highest confidence
            best_box = pred_boxes[np.argmax(confidences)]
            iou_value = calculate_iou(true_box, best_box)
        
        # Show result
        plt.figure(figsize=(10, 8))
        plt.imshow(result_img)
        plt.title(f"IoU: {iou_value:.4f}, Plate: {row['plate_text']}")
        plt.axis('off')
        plt.show()

# Test the detector
print("Testing detection on sample images...")
test_detection_on_samples(yolo_model, df)

In [ ]:
# Comprehensive evaluation to match the metrics used in your previous model
def evaluate_license_plate_detection_yolo(model, df, num_samples=5):
    """
    Evaluate YOLO model on the dataset with metrics matching the previous approach
    """
    # Calculate IoU for all samples in the dataset
    iou_values = []
    pred_boxes_list = []
    confidences_list = []
    
    for idx, row in tqdm(df.iterrows(), total=len(df), desc="Evaluating model"):
        img_path = row["image_path"]
        
        # Ground truth box (x1, y1, x2, y2 format)
        true_box = [row["x"], row["y"], row["x"] + row["w"], row["y"] + row["h"]]
        
        # Run detection
        _, pred_boxes, confidences = detect_license_plate_yolo(model, img_path)
        
        # Calculate IoU if any detection is found
        if len(pred_boxes) > 0:
            # Get the box with highest confidence
            best_pred_idx = np.argmax(confidences)
            best_box = pred_boxes[best_pred_idx]
            best_conf = confidences[best_pred_idx]
            
            iou = calculate_iou(true_box, best_box)
            iou_values.append(iou)
            pred_boxes_list.append(best_box)
            confidences_list.append(best_conf)
        else:
            # No detection
            iou_values.append(0.0)
            pred_boxes_list.append([0, 0, 0, 0])
            confidences_list.append(0.0)
    
    # Find best and worst predictions
    iou_indices = np.argsort(iou_values)
    worst_indices = iou_indices[:num_samples//2]
    best_indices = iou_indices[-num_samples//2:]
    
    # Visualization of best and worst cases
    plt.figure(figsize=(15, 4*num_samples))
    
    samples_to_show = np.concatenate([worst_indices, best_indices])
    
    for i, idx in enumerate(samples_to_show):
        row = df.iloc[idx]
        img_path = row["image_path"]
        img = cv2.imread(img_path)
        img_rgb = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        
        # Draw ground truth (green)
        x, y, w, h = row["x"], row["y"], row["w"], row["h"]
        cv2.rectangle(img_rgb, (x, y), (x + w, y + h), (0, 255, 0), 2)
        
        # Draw prediction (red) if available
        if len(pred_boxes_list) > idx:
            pred_box = pred_boxes_list[idx]
            if not np.all(pred_box == 0):
                x1, y1, x2, y2 = map(int, pred_box)
                cv2.rectangle(img_rgb, (x1, y1), (x2, y2), (255, 0, 0), 2)
        
        plt.subplot(num_samples, 2, i+1)
        plt.imshow(img_rgb)
        plt.title(f"IoU: {iou_values[idx]:.4f} {'(Worst)' if idx in worst_indices else '(Best)'}", fontsize=10)
        plt.axis('off')
    
    plt.tight_layout()
    plt.show()
    
    # Calculate area of each license plate for size categorization
    def calculate_normalized_area(row):
        # Get image dimensions
        img = cv2.imread(row["image_path"])
        if img is None:
            return 0
        
        h, w = img.shape[:2]
        # Calculate normalized area
        return (row["w"] * row["h"]) / (w * h)
    
    # Get normalized areas
    areas = [calculate_normalized_area(df.iloc[i]) for i in range(len(df))]
    
    # Define thresholds (matching your previous notebook)
    small_threshold = 0.03
    large_threshold = 0.1
    
    # Categorize plates
    size_categories = []
    for area in areas:
        if area < small_threshold:
            size_categories.append(0)  # Small
        elif area > large_threshold:
            size_categories.append(2)  # Large
        else:
            size_categories.append(1)  # Medium
    
    # Group by plate size
    small_ious = [iou for iou, cat in zip(iou_values, size_categories) if cat == 0]
    medium_ious = [iou for iou, cat in zip(iou_values, size_categories) if cat == 1]
    large_ious = [iou for iou, cat in zip(iou_values, size_categories) if cat == 2]
    
    # Print statistics (same format as your previous notebook)
    print("Overall Performance:")
    print(f"Average IoU: {np.mean(iou_values):.4f}")
    print(f"Median IoU: {np.median(iou_values):.4f}")
    print(f"Min IoU: {np.min(iou_values):.4f}")
    print(f"Max IoU: {np.max(iou_values):.4f}")
    print("\nPerformance by Plate Size:")
    print(f"Small Plates: Avg IoU = {np.mean(small_ious) if small_ious else 0:.4f}, Count = {len(small_ious)}")
    print(f"Medium Plates: Avg IoU = {np.mean(medium_ious) if medium_ious else 0:.4f}, Count = {len(medium_ious)}")
    print(f"Large Plates: Avg IoU = {np.mean(large_ious) if large_ious else 0:.4f}, Count = {len(large_ious)}")
    
    # Plot IoU distribution (same format as your previous notebook)
    plt.figure(figsize=(15, 6))
    
    # Histogram of IoU values
    plt.subplot(1, 2, 1)
    plt.hist(iou_values, bins=20, alpha=0.7, color='blue')
    plt.axvline(np.mean(iou_values), color='red', linestyle='dashed', linewidth=2, label=f'Mean IoU: {np.mean(iou_values):.4f}')
    plt.axvline(np.median(iou_values), color='green', linestyle='dashed', linewidth=2, label=f'Median IoU: {np.median(iou_values):.4f}')
    plt.title('IoU Distribution')
    plt.xlabel('IoU Value')
    plt.ylabel('Count')
    plt.legend()
    
    # IoU by plate size
    plt.subplot(1, 2, 2)
    boxplot_data = [small_ious, medium_ious, large_ious]
    plt.boxplot(boxplot_data, labels=['Small', 'Medium', 'Large'])
    plt.title('IoU by License Plate Size')
    plt.ylabel('IoU Value')
    plt.xlabel('Plate Size')
    plt.grid(True, linestyle='--', alpha=0.5)
    
    plt.tight_layout()
    plt.show()
    
    return iou_values

# Evaluate on a subset for performance
test_df = df.sample(n=min(100, len(df)), random_state=42)
print("Running comprehensive evaluation...")
iou_values = evaluate_license_plate_detection_yolo(yolo_model, test_df, num_samples=6)

# Comparing YOLO vs. Custom CNN Results

The above results from the YOLO-based model can be directly compared with your previous custom CNN approach. YOLO typically provides significant improvements due to several factors:

1. **Better Feature Extraction**: YOLO uses a more sophisticated backbone network pre-trained on millions of images

2. **Scale Invariance**: YOLO's feature pyramid design helps it detect license plates of varying sizes

3. **Contextual Understanding**: YOLO can use surrounding vehicle information to better locate license plates

4. **Robustness**: Better handling of challenging conditions like poor lighting and partial occlusion

5. **Confidence Scores**: YOLO provides detection confidence, allowing you to filter unreliable detections

The metrics we used for evaluation are directly comparable to your previous approach, making it easy to quantify the improvement.

In [ ]:
# Utility function for real-world license plate detection
def detect_and_extract_license_plate(model, image_path, conf_threshold=0.25):
    """
    Complete function to detect and extract license plates from an image.
    Returns the cropped license plate region for further processing.
    """
    # Load the image
    img = cv2.imread(image_path)
    if img is None:
        print(f"Could not read image at {image_path}")
        return None, None
    
    img_rgb = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    
    # Run inference
    results = model.predict(img_rgb, conf=conf_threshold, verbose=False)[0]
    
    # Get detection boxes and confidences
    boxes = results.boxes.xyxy.cpu().numpy()  # [x1, y1, x2, y2] format
    confidences = results.boxes.conf.cpu().numpy()
    
    if len(boxes) == 0:
        print("No license plate detected")
        return img_rgb, None
    
    # Get the highest confidence detection
    best_idx = np.argmax(confidences)
    best_box = boxes[best_idx]
    best_conf = confidences[best_idx]
    
    # Extract coordinates
    x1, y1, x2, y2 = map(int, best_box)
    
    # Draw on image
    result_img = img_rgb.copy()
    cv2.rectangle(result_img, (x1, y1), (x2, y2), (0, 255, 0), 2)
    cv2.putText(result_img, f"{best_conf:.2f}", (x1, y1-10),
               cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 2)
    
    # Extract the plate region
    plate_region = img_rgb[y1:y2, x1:x2]
    
    # Display results
    plt.figure(figsize=(12, 6))
    
    plt.subplot(1, 2, 1)
    plt.imshow(result_img)
    plt.title(f"License Plate Detection (Conf: {best_conf:.2f})")
    plt.axis('off')
    
    plt.subplot(1, 2, 2)
    plt.imshow(plate_region)
    plt.title("Extracted License Plate")
    plt.axis('off')
    
    plt.tight_layout()
    plt.show()
    
    return result_img, plate_region

# Test the complete detection pipeline on a few images
test_images = random.sample(list(df["image_path"]), 3)

for img_path in test_images:
    print(f"\nProcessing image: {os.path.basename(img_path)}")
    _, plate = detect_and_extract_license_plate(yolo_model, img_path)